In [186]:
import requests
from lxml import html
import time
import collections
import pandas as pd
from openpyxl import load_workbook

In [187]:
send_headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36",
        "Connection": "keep-alive",
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8",
        "Accept-Language": "zh-CN,zh;q=0.8"}

# 获取所有目标子页面url

In [190]:
target_html_dict = collections.OrderedDict()
BASE_URL = 'https://wsjkw.sh.gov.cn'
for i in range(1,16):
    time.sleep(1)
    page = '_'+str(i)
    if i == 1:
        page = ''
    url = 'https://wsjkw.sh.gov.cn/xwfb/index{}.html'.format(page)
    response = requests.get(url,headers=send_headers).text
    # print(response)
    selector = html.fromstring(response)
    urls = selector.xpath("//div[@class='main-container margin-top-15']//ul[@class='uli16 nowrapli list-date ']/li")
    for url in urls:
        title = url.xpath('a/@title')[0]
        herf = url.xpath('a/@href')[0]
        if '（0-24时）' in title:
            print(title)  
            # print(BASE_URL + herf)
            if herf[:4] == 'http':
                target_html_dict[title] = herf
            else:
                target_html_dict[title] = BASE_URL + herf
# for url in target_html_dict.items():
#     print(url)

4月18日（0-24时）本市各区确诊病例、无症状感染者居住地信息
4月17日（0-24时）本市各区确诊病例、无症状感染者居住地信息
4月16日（0-24时）本市各区确诊病例、无症状感染者居住地信息
4月15日（0-24时）本市各区确诊病例、无症状感染者居住地信息
4月14日（0-24时）本市各区确诊病例、无症状感染者居住地信息
4月13日（0-24时）本市各区确诊病例、无症状感染者居住地信息
【最新】4月12日（0-24时）本市各区确诊病例、无症状感染者居住地信息
4月11日（0-24时）本市各区确诊病例、无症状感染者居住地信息
4月10日（0-24时）本市各区确诊病例、无症状感染者居住地信息
4月9日（0-24时）本市各区确诊病例、无症状感染者居住地信息
4月8日（0-24时）本市各区确诊病例、无症状感染者居住地信息
4月7日（0-24时）本市各区确诊病例、无症状感染者居住地信息
4月6日（0-24时）本市各区确诊病例、无症状感染者居住地信息
4月5日（0-24时）本市各区确诊病例、无症状感染者居住地信息
4月4日（0-24时）本市各区确诊病例、无症状感染者居住地信息
4月3日（0-24时）本市各区确诊病例、无症状感染者居住地信息
4月2日（0-24时）本市各区确诊病例、无症状感染者居住地信息
4月1日（0-24时）本市各区确诊病例、无症状感染者居住地信息
3月31日（0-24时）本市各区确诊病例、无症状感染者居住地信息
3月30日（0-24时）本市各区确诊病例、无症状感染者居住地信息
3月29日（0-24时）本市各区确诊病例、无症状感染者居住地信息
3月28日（0-24时）本市各区确诊病例、无症状感染者居住地信息
3月26日（0-24时）本市各区确诊病例、无症状感染者居住地信息
3月25日（0-24时）本市各区确诊病例、无症状感染者居住地信息
3月24日（0-24时）本市各区确诊病例、无症状感染者居住地信息
3月23日（0-24时）本市各区确诊病例、无症状感染者居住地信息
3月22日（0-24时）本市各区确诊病例、无症状感染者居住地信息
3月21日（0-24时）本市各区确诊病例、无症状感染者居住地信息
3月20日（0-24时）本市各区确诊病例、无症状感染者居住地信息
3月19日（0-24时）本市各区确诊病例、无症状感染者居住地信息
3月18日（0-24时）本市各

从上面分析来看 网址以4.5为界限，有普通的网页，以及微信网页两种

所以分别需要写两种对应的方法

# 普通子网页爬取方法

In [191]:
addr_df = pd.DataFrame()
last = ['村','弄','号','园','区','宅','沿']
def getNormalPage(date, url):
    addr = []
    response = requests.get(url,headers=send_headers).text
    selector = html.fromstring(response)
    contents = selector.xpath("//div[@class='main-container margin-top-15']//div[@class='Article_content']/p")

    for content in contents:
        possibleLoc = content.xpath('span/span/text()')
        loc = content.xpath('span/text()')
        if loc != []:
            # print(loc[0][-1:] in last)
            if possibleLoc != []:
                loc[0] = possibleLoc[0]+loc[0]
        
            if loc[0][-1:] == '，' or loc[0][-1:] == '：':
                # loc = loc[0].replace(',',' ')
                addr.append(loc[0][:-1])
            elif loc[0][-1:] == '。' and loc[0][-1:] in last:
                addr.append(loc[0][:-1])
    return (date,addr)

# 微信子网页爬取方法

In [192]:
def getWxPage(date, url):
    addr = []
    response = requests.get(url,headers=send_headers).text
    selector = html.fromstring(response)
    selector = html.fromstring(response)
    contents = selector.xpath("//div[@class='rich_media_inner']//div[@class='rich_media_area_primary_inner']//div[@class='rich_media_wrp']//div[@id='js_content']//section[@data-id='72469']//section[@data-autoskip='1']/p")

    for content in contents:
        loc = content.xpath('span/text()')
        if loc != []:
            # print(loc[0][-1:] in last)
            if loc[0][-1:] == '，' or loc[0][-1:] == '：':
                # loc = loc[0].replace(',',' ')
                addr.append(loc[0][:-1])
            elif loc[0][-1:] == '。' and loc[0][-1:] in last:
                addr.append(loc[0][:-1])
    return (date,addr)

In [193]:
wb = load_workbook('./data.xlsx')
excel_writer = pd.ExcelWriter('data.xlsx',engine='openpyxl')
# print(excel_writer.path)
excel_writer.book = wb

for title,url in target_html_dict.items():
    # print(url[8:17])
    if url[8:17] == 'mp.weixin':
    # if url == 'https://mp.weixin.qq.com/s/vxFiV2HeSvByINUlTmFKZA':
        res = getWxPage(date=title,url=url)
    # elif url == 'https://wsjkw.sh.gov.cn/xwfb/20220321/2cda1b24b5304d118352bdfd32af0aa4.html':
    else:
        res = getNormalPage(date=title,url=url)
    
    _t_df = pd.DataFrame()
    _t_df['地址'] = res[1]
    _t_df.to_excel(excel_writer, sheet_name=res[0],)

excel_writer.save() # 储存文件 
excel_writer.close() 
    

/usr/local/lib/python3.8/site-packages/openpyxl/workbook/child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")
